In [4]:
import pandas as pd
import json
import os
#import mysql.connector

#This is to direct the path to get the data as states

path = '../pulse/data/map/transaction/hover/country/india/state'
Agg_state_list=os.listdir(path)
Agg_state_list
#Agg_state_list--> to get the list of states in India

#<------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------>#

#This is to extract the data's to create a dataframe

clm={'State':[],'Year':[],'Quater':[],'State_or_District_name':[], 'Transacion_type':[], 'Transaction_count':[], 'Transacion_amount':[]}

for i in Agg_state_list:
    #p_i=os.path+i+"/"
    p_i=os.path.join(path, i)
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        #p_j=p_i+j+"/"
        p_j=os.path.join(p_i, j)
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            # p_k=p_j+k
            # Data=open(p_k,'r')
            p_k = os.path.join(p_j, k)
            #D=json.load(Data)
            with open(p_k, 'r') as Data:
                D = json.load(Data)
            for z in D['data']['hoverDataList']:
              Name=z['name']
              type_=z['metric'][0]['type']
              count=z['metric'][0]['count']
              amount=z['metric'][0]['amount']
              clm['State_or_District_name'].append(Name)
              clm['Transacion_type'].append(type_)
              clm['Transaction_count'].append(count)
              clm['Transacion_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_Trans=pd.DataFrame(clm)
print(Agg_Trans)


                           State  Year  Quater  \
0      andaman-&-nicobar-islands  2018       1   
1      andaman-&-nicobar-islands  2018       1   
2      andaman-&-nicobar-islands  2018       1   
3      andaman-&-nicobar-islands  2018       2   
4      andaman-&-nicobar-islands  2018       2   
...                          ...   ...     ...   
20599                west-bengal  2024       4   
20600                west-bengal  2024       4   
20601                west-bengal  2024       4   
20602                west-bengal  2024       4   
20603                west-bengal  2024       4   

                  State_or_District_name Transacion_type  Transaction_count  \
0      north and middle andaman district           TOTAL                442   
1                 south andaman district           TOTAL               5688   
2                      nicobars district           TOTAL                528   
3      north and middle andaman district           TOTAL                825   
4   

In [5]:
import pymysql
conn_pymysql = pymysql.connect(
    host="localhost",
    user="root",
    password="root123",
)
cursor_pymysql = conn_pymysql.cursor()
print("PyMySQL connection established!")

PyMySQL connection established!


In [6]:
cursor_pymysql.execute("""USE PHONEPE_INSURANCE;""")
cursor_pymysql.execute("""CREATE TABLE map_transactions_district(State varchar(256),Year int,Quater int,State_or_District_name varchar(256), Transaction_type varchar(256), Transaction_count bigint, Transaction_amount bigint);""")
print('Table created!')

OperationalError: (1050, "Table 'map_transactions_district' already exists")

In [7]:
data_list = Agg_Trans.values.tolist()
query = """
    INSERT INTO map_transactions_district(State,Year,Quater,State_or_District_name, Transaction_type, Transaction_count, Transaction_amount)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
"""
cursor_pymysql.executemany(query, data_list)
conn_pymysql.commit()
print("Data inserted using to_list()")

Data inserted using to_list()
